# 1. MultiThread for Crawl User_info from list of UserURLs

In [1]:
%cd ../src

c:\Users\LENOVO\Desktop\facebook_crawler\src


In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
import random
import threading
from utils.login import login
import user_profile

def initialize_driver():
    option = Options()
    option.add_argument("--disable-infobars")
    option.add_argument("start-maximized")
    option.add_argument("--disable-extensions")
    option.add_argument("--incognito")
    option.add_experimental_option(
        "prefs", {"profile.default_content_setting_values.notifications": 1}
    )
    return webdriver.Chrome(options=option)

def crawl_user_info(driver, user_list, start_index, chunk_size, result):
    user_crawl =  user_profile.UserInfo(driver)
    for i in range(start_index, start_index+chunk_size):
        user_info = user_crawl.get_user_info(user_list[i])
        result[i] = user_info


username = "yelan482@gmail.com"
password = 'Linhcute542002'

no_thread = 3
base_url = 'https://www.facebook.com/'
drivers = [initialize_driver() for _ in range(no_thread)]

for driver in drivers:
    driver.get(base_url)
    login(driver, username, password)
    time.sleep(3)



def crawl_info_multithread(drivers, user_list, no_thread):
    result = [None] * len(user_list)
    chunk_size = len(user_list) // no_thread
    start = 0
    threads = []
    for driver in drivers:
        thread = threading.Thread(target=crawl_user_info, args=(driver, user_list, start, chunk_size, result))
        threads.append(thread)
        start += chunk_size
    start_time = time.perf_counter()
    for thread in threads:
        thread.start()
    for thread in threads:
        thread.join()
    print(time.perf_counter() - start_time, 'seconds')
    return result

17:05:31 utils.Login: Login Successfull
17:05:31 utils.Login: Login Successfull
17:05:31 utils.Login: Login Successfull


In [3]:
columns = ['url','name', 'Gender', 'Birthdate', 'Birthyear', 'Highschool', 'College', 'Currentcity', 'Hometown', 'group']

def merge_dict(raw_data_dict, group, url):
    name, basic_inf, school, locate =  raw_data_dict
    raw_data = {'url':url, 'name':name, 'group': group , **basic_inf, **school, **locate}
    data = {}
    keys = raw_data.keys()
    for i in columns:
        if i in keys:
            i = i.replace(' ','')
            if i=='Highschool' or i=='College':
                data[i] = raw_data[i].replace('Studied at ', '').replace('Went to ', '')
            else:
                data[i] = raw_data[i]
        else:
            data[i] = None
    return data

In [6]:
import pandas as pd
start = 0
no_to_crawl = 43

df = pd.read_csv('../datasets/data.csv')
df = df[start:start+no_to_crawl]

no_thread = 3
batch_size = 30
save_path = '../datasets/user_info_from_group.csv'

for _ in range(no_to_crawl // batch_size):
    user_list = df['member_url'][start:start+batch_size].to_list()
    result = crawl_info_multithread(drivers, user_list, no_thread)
    print(start,start+batch_size)
    data = []
    chunk_df = df[start:start+batch_size]
    for i in range(start,start+len(result)):
        group = chunk_df['group'][i]
        url = chunk_df['member_url'][i]
        print(group,url)
        try:
            new_dict = merge_dict(result[i], group, url)
            data.append(new_dict)
        except:
            continue
    crawl_df = pd.DataFrame(data)
    existing_df = pd.read_csv(save_path)
    existing_df = pd.concat([existing_df, crawl_df], ignore_index=True)
    existing_df.to_csv(save_path, index=False)
    start += batch_size

if no_to_crawl % batch_size != 0:
    user_list = df['member_url'][start:].to_list()
    result = crawl_info_multithread(drivers, user_list, no_thread)
    data = []
    chunk_df = df[start:]
    for i in range(start,start+len(result)):
        group = chunk_df['group'][i]
        url = chunk_df['member_url'][i]
        print(group,url)
        try:
            new_dict = merge_dict(result[i], group, url)
            data.append(new_dict)
        except:
            continue
    crawl_df = pd.DataFrame(data)
    existing_df = pd.read_csv(save_path)
    existing_df = pd.concat([existing_df, crawl_df], ignore_index=True)
    existing_df.to_csv(save_path, index=False)

167.83690649999997 seconds
0 30
AN LUONG DONG HIGH SCHOOL https://www.facebook.com/profile.php?id=100053397864324
AN LUONG DONG HIGH SCHOOL https://www.facebook.com/profile.php?id=100052716540727
AN LUONG DONG HIGH SCHOOL https://www.facebook.com/profile.php?id=100003333302732
AN LUONG DONG HIGH SCHOOL https://www.facebook.com/profile.php?id=100002710709082
AN LUONG DONG HIGH SCHOOL https://www.facebook.com/profile.php?id=100009002575989
AN LUONG DONG HIGH SCHOOL https://www.facebook.com/profile.php?id=100010871461125
AN LUONG DONG HIGH SCHOOL https://www.facebook.com/profile.php?id=100009175189050
AN LUONG DONG HIGH SCHOOL https://www.facebook.com/profile.php?id=100022061332007
AN LUONG DONG HIGH SCHOOL https://www.facebook.com/profile.php?id=100007735600408
AN LUONG DONG HIGH SCHOOL https://www.facebook.com/profile.php?id=100006422435007
AN LUONG DONG HIGH SCHOOL https://www.facebook.com/profile.php?id=100000606050454
AN LUONG DONG HIGH SCHOOL https://www.facebook.com/profile.php?id=

# 2. MultiThread for Crawl multiple group members from list of groups

In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
import random
import threading
from utils.login import login
import user_profile
import group

def initialize_driver():
    option = Options()
    option.add_argument("--disable-infobars")
    option.add_argument("start-maximized")
    option.add_argument("--disable-extensions")
    option.add_experimental_option(
        "prefs", {"profile.default_content_setting_values.notifications": 1}
    )
    return webdriver.Chrome(options=option)

def crawl_group_member(driver, group_list, start_index, chunk_size, result):
    group_crawler = group.Group(browser=driver)
    for i in range(start_index, start_index+chunk_size):
        try:
            user_list = group_crawler.get_url_from_members(group_list[i], max_user=100)
        except:
            user_list = None
        result[i] = user_list


username = "lth2112002@gmail.com"
password = 'Baoanh123'


group_links=[
    'https://www.facebook.com/groups/932212904285349',
    'https://www.facebook.com/groups/1986195691670003',
    'https://www.facebook.com/groups/263503162332380',
    'https://www.facebook.com/groups/flexdenhoithocuoicung',
    'https://www.facebook.com/groups/nthehatoansongsong',
]

base_url = 'https://www.facebook.com/'
drivers = [initialize_driver() for _ in range(5)]

for driver in drivers:
    driver.get(base_url)
    login(driver, username, password)

result = [None] * len(group_links)

chunk_size = len(group_links) // 5   # Max = 5
start = 0

threads = []
for driver in drivers:
    thread = threading.Thread(target=crawl_group_member, args=(driver, group_links, start, chunk_size, result))
    threads.append(thread)
    start += chunk_size

start_time = time.perf_counter()

for thread in threads:
    thread.start()

for thread in threads:
    thread.join()

print(time.perf_counter() - start_time, 'seconds')

for driver in drivers:
    driver.quit()

Success Login
Success Login
Success Login
Success Login
Success Login
Start Crawling User from Group: CLB Sáng tạo trẻ - Trường THPT A Lưới
===73/100
===93/100
===103/100


100%|██████████| 100/100 [00:01<00:00, 62.72it/s]


20.2755345999999 seconds


In [7]:
result

[None,
 ['https://www.facebook.com/profile.php?id=100071684331377',
  'https://www.facebook.com/profile.php?id=100015733534289',
  'https://www.facebook.com/profile.php?id=100013886446373',
  'https://www.facebook.com/profile.php?id=61555335785751',
  'https://www.facebook.com/profile.php?id=100092380986251',
  'https://www.facebook.com/profile.php?id=100082605951622',
  'https://www.facebook.com/profile.php?id=100003046968242',
  'https://www.facebook.com/profile.php?id=100003677062318',
  'https://www.facebook.com/profile.php?id=100008105701061',
  'https://www.facebook.com/profile.php?id=100066320130406',
  'https://www.facebook.com/profile.php?id=100077485805711',
  'https://www.facebook.com/profile.php?id=61551622838976',
  'https://www.facebook.com/profile.php?id=100076634932874',
  'https://www.facebook.com/profile.php?id=100084516473525',
  'https://www.facebook.com/profile.php?id=100084144864498',
  'https://www.facebook.com/profile.php?id=100031024537497',
  'https://www.face

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
import random
import threading
from utils.login import login
import user_profile
import group

username = "lth2112002@gmail.com"
password = 'Baoanh123'


group_links=[
'https://www.facebook.com/groups/932212904285349',
'https://www.facebook.com/groups/1986195691670003',
    'https://www.facebook.com/groups/263503162332380',
    'https://www.facebook.com/groups/flexdenhoithocuoicung',
    'https://www.facebook.com/groups/nthehatoansongsong',
]

base_url = 'https://www.facebook.com/'

def initialize_driver():
    option = Options()
    option.add_argument("--disable-infobars")
    option.add_argument("start-maximized")
    option.add_argument("--disable-extensions")
    option.add_experimental_option(
        "prefs", {"profile.default_content_setting_values.notifications": 1}
    )
    return webdriver.Chrome(options=option)

def crawl_group_member(driver, group_list, start_index, chunk_size, result):
    group_crawler = group.Group(browser=driver)
    for i in range(start_index, start_index+chunk_size):
        try:
            user_list = group_crawler.get_url_from_members(group_list[i], max_user=100)
        except:
            user_list = None
        result[i] = user_list

def multi_thread_crawl(group_links,num_thread=5):

    drivers = [initialize_driver() for _ in range(5)]

    for driver in drivers:
        driver.get(base_url)
        login(driver, username, password)

    result = [None] * len(group_links)
    chunk_size = len(group_links) //  num_thread  # Max = 5
    start = 0
    threads = []
    for driver in drivers:
        thread = threading.Thread(target=crawl_group_member, args=(driver, group_links, start, chunk_size, result))
        threads.append(thread)
        start += chunk_size

    start_time = time.perf_counter()
    for thread in threads:
        thread.start()
    for thread in threads:
        thread.join()
    print(time.perf_counter() - start_time, 'seconds')

    for driver in drivers:
        driver.quit()

    return result